# Multiclass SVM
Unter der Verwendung des *One to Rest Approaches*

https://www.baeldung.com/cs/svm-multiclass-classification

Horea Muresan, Mihai Oltean, Fruit recognition from images using deep learning, Acta Univ. Sapientiae, Informatica Vol. 10, Issue 1, pp. 26-42, 2018.


In [1]:
# Datenverarbeitung und Dateioperationen
import glob
import os
import cv2

# Wissenschaftliche Berechnungen und Hilfsfunktionen
import numpy as np
import matplotlib.pyplot as plt

# Maschinelles Lernen und Modellierung
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Leistungsbewertung
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

In [2]:
def getAllData(data_type, dim, print_summary=False):
    images = []
    labels = []

    # Basispfad für das Verzeichnis der Daten
    base_path = "frutis_360/fruits-360_dataset/fruits-360/LessData/"

    # Funktion, um alle Unterordner (Datenkategorien) zu erhalten
    def get_categories(path):
        return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    categories = get_categories(os.path.join(base_path, data_type))
    total_images = 0  

    for i, category in enumerate(categories):
        category_path = os.path.join(base_path, data_type, category)
        for image_path in glob.glob(os.path.join(category_path, "*.jpg")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (dim, dim))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            images.append(image)
            labels.append(i)
            total_images += 1

    if print_summary:
        print(f"Found {total_images} images belonging to {len(categories)} classes.")

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [3]:
dim = 100

# Abrufen der Daten für Training und Test, mit der Zusammenfassung
print("Training: ")
X_train, y_train = getAllData('Training', dim, print_summary=True)
print("Test: ")
X_test, y_test = getAllData('Test', dim, print_summary=True)

Training: 
Found 41310 images belonging to 81 classes.
Test: 
Found 13873 images belonging to 81 classes.


In [4]:
#Skalierung der Daten
scaler = StandardScaler()
X_train = scaler.fit_transform([i.flatten() for i in X_train])
X_test = scaler.fit_transform([i.flatten() for i in X_test])

## SVM - One-to-Rest
Der One-to-Rest approach nutzt $m$ SVMs. Jede SVM wird eine der $m$ Klassen vorhersagen.
- Hier sind es $m = 81$ SVMs


Der One-to-One approach nutzt $\frac{m (m-1)}{2}$ SVMs. Den jede Klasse wird zu einander in Relation gesetzt.
- Hier sind es $\frac{81 (81-1)}{2} = 3240$ SVMs die berechnet werden müssten.

Initialisieren des SVM-Klassifikatoren:

In [5]:
# Initialisieren des SVM-Klassifikators mit one to rest
model_ovr = SVC(decision_function_shape='ovr')

In [6]:
param_grid = {
    'C': [0.1, 1, 10],  # Werte für den Regularisierungsparameter
    'gamma': [1, 0.1, 0.01],  # Werte für den Gamma-Parameter (für nichtlineare Kerne)
    'kernel': ['poly']  # Kernelvarianten
}

In [7]:
gridsearchcv = GridSearchCV(model_ovr, param_grid=param_grid, cv=3, verbose=2)

In [ ]:
gridsearchcv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .......................C=0.1, gamma=1, kernel=poly; total time=108.8min


In [ ]:
print("Beste Parameter:", gridsearchcv.best_params_)
model = gridsearchcv.best_estimator_

In [ ]:
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Testgenauigkeit:", test_accuracy)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))
poly_accuracy = accuracy_score(y_test, y_pred)
poly_f1 = f1_score(y_test, y_pred, average='weighted')
print('Accuracy (Polynomial Kernel): {:.2f}%'.format(poly_accuracy*100))
print('F1 (Polynomial Kernel): {:.2f}'.format(poly_f1*100))

In [ ]:
# Berechnung der Confusion Matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

# Visualisierung der Confusion Matrix
plt.figure(figsize=(20,20))
plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()

# Beschriftung der Achsen und Titel
plt.xlabel('Vorhergesagte Klasse')
plt.ylabel('Tatsächliche Klasse')
plt.show()

---

## Export the SVM (OVR) Model

In [ ]:
from joblib import dump

In [ ]:
dump(model, 'svm_multi_classifier.joblib')